# Testing deepseek models

Preprocessing

In [1]:
from pdf_parser import *
import os

pdf_folder_path = "../data/PDF"
# pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
pdf_files = ["24EM03456.pdf"]
image_save_folder_path = "../data/images"

pdf_text_image = {}
for pdf_file in pdf_files:
    pdf_text_image[os.path.splitext(pdf_file)[0]] = {"text":extract_all(os.path.join(pdf_folder_path,pdf_file))}
    pdf_images = extract_pdf2image(f"{pdf_folder_path}/{pdf_file}")
    pdf_text_image[os.path.splitext(pdf_file)[0]]["image"] = pdf_images

print(pdf_text_image.keys())
print(pdf_text_image)

with open("../prompt/system_prompt_metadata.txt", "r") as f:
    system_prompt = f.read()
print(system_prompt)

2025-05-22 14:59:36,743 CropBox missing from /Page, defaulting to MediaBox
2025-05-22 14:59:37,593 CropBox missing from /Page, defaulting to MediaBox
2025-05-22 14:59:38,303 CropBox missing from /Page, defaulting to MediaBox


dict_keys(['24EM03456'])
{'24EM03456': {'text': "COPIE INTERNE 29/10/2024\n\nDr ROGER Thierry \nCHIREC DELTA\nRadiodiagnostic\nBOULEVARD DU TRIOMPHE 1\n1160 AUDERGHEM\n\nCentre d’Anatomie \n Pathologique H.U.B.\n\nRue Meylemeersch 90 - 1070 Anderlecht \nMijlemeerschstraat 90 – 1070 Anderlecht\n\nPATIENT :    \n  \nID :   \nRéf. Externe : 24CU052383 \n \nPrélevé le 30/07/2024 à 30/07/2024 11:30 \nReçu le 08/08/2024\n\nEXAMEN : 24EM03456 \n \nPrescripteur : Dr ROGER Thierry\n\nDirectrice de Service \nPr Myriam Remmelink\n\nEquipe Médicale  \nDr Nicolas de Saint Aubain\n\nPr Nicky D’Haene \nDr Maria Gomez Galdon\n\nDr Chirine Khaled \nPr Denis Larsimont\n\nDr Laetitia Lebrun\n\nDr Calliope Maris \nPr Jean-Christophe Noël\n\nPr Isabelle Salmon \nDr Anne-Laure Trépant\n\nPr Laurine Verset\n\nRECHERCHE PAR « NEXT GENERATION SEQUENCING » DE \nMUTATIONS DANS 25 GENES IMPLIQUES DANS LES CANCERS\n\nConsultant (e) s\n\nDr Sarah Bouri \nDr Xavier Catteau \nDr Roland de Wind\n\nPULMONAIRES, LES GIS

### Deepseek R1 1.5B distilled Qwen
Not yet supported via the hugging face transformers library.

In [2]:
from transformers import pipeline
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

pipe = pipeline(
    "text-generation", #"image-text-to-text"
    model=model_name,
    torch_dtype=torch.bfloat16,
    device="cpu", #uses "cpu" here because the "cuda" requires 3 Go more of VRAM (5GO total)
)
print("pipeline initialized")

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": (
            [{"type": "text", "text": pdf_text_image["24EM03456"]["text"]}]
        )
    }
]

output = pipe(messages, max_new_tokens=250) #temperature=0
print(output[0]["generated_text"][-1]["content"])

/home/alexa/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-22 14:59:44.780842: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 14:59:45.183062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747918785.407918   18203 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747918785.465506   18203 cuda_blas.cc:1407] Unable to r

pipeline initialized


TypeError: can only concatenate str (not "list") to str

In [ ]:
#save the output in json format
import json
import pandas as pd

answer = output[0]["generated_text"][-1]["content"]
cleaned = answer.replace("```json", "").replace("```", "").strip() #clean the answer
row = json.loads(cleaned)
print(row)

df = pd.DataFrame([row])
#save to xlsx
df.to_excel("../out/metadata_deepseek_R1_1.5B.xlsx", index=False)